In [1]:
from src.finetuner import FineTuner
from src.preprocessor import Preprocessor
from transformers import (
    T5ForConditionalGeneration,
    #BartForConditionalGeneration,
    AutoTokenizer,
    Seq2SeqTrainingArguments
)
from datasets import load_dataset
#from indobenchmark import IndoNLGTokenizer

c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


# Dataset

In [2]:
#constant
max_length = 30
text_col = 'content'
label_col = 'postprocess_quintuplet'
preprocess_type = 'p02'
experiment_type = 'p22'
SAVE_PATH = f'../models/{experiment_type}_quintuplet'
PRETRAINED_MODEL = "../models/pt-indot5-MLM_TA_PT"#"indobenchmark/indobart-v2"#"Wikidepia/IndoT5-base"#"../models/pt-indot5-MLM_PT"  
TOKENIZER_PATH = "Wikidepia/IndoT5-base" #"indobenchmark/indobart-v2"
DATA_PATH = '../Data/quintuplet/quintuplet_postprocessed.csv'

In [3]:
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_PATH)
#tokenizer = IndoNLGTokenizer.from_pretrained(TOKENIZER_PATH)
preprocessor = Preprocessor(preprocess_type, tokenizer, max_length, text_col, label_col)

In [4]:
# raw_dataset = load_dataset('csv', data_files={
#     'train' : f'{DATA_PATH}_train.csv',
#     'test' : f'{DATA_PATH}_test.csv',
#     'val' : f'{DATA_PATH}_val.csv'
# })
raw_dataset = load_dataset('csv', data_files=DATA_PATH)
#raw_dataset = raw_dataset.filter(lambda x: x['is_comparative']==False)
splitted_dataset = raw_dataset['train'].train_test_split(test_size=100, seed=42)
tokenized_dataset = splitted_dataset.map(preprocessor.preprocess_dataset, batched=True, remove_columns=raw_dataset['train'].column_names)

Found cached dataset csv (C:/Users/danendra/.cache/huggingface/datasets/csv/default-7c3f713988208aa5/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 399.23it/s]
Loading cached split indices for dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-7c3f713988208aa5\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-06cdaa4abbd48a43.arrow and C:\Users\danendra\.cache\huggingface\datasets\csv\default-7c3f713988208aa5\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-475e0560816dc728.arrow
Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-7c3f713988208aa5\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-294dd2bf963c7f67.arrow
Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-7c3f713988208aa5\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853b

In [5]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1894
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 100
    })
})

In [6]:
len_labels = []
len_inputs = []
for i in range(len(tokenized_dataset['train'])):
    len_labels.append(len(tokenized_dataset['train'][i]['labels']))
    len_inputs.append(len(tokenized_dataset['train'][i]['input_ids']))

In [7]:
sum(len_labels)/len(len_labels), sum(len_inputs)/len(len_inputs)

(97.55227032734952, 30.0)

In [8]:
splitted_dataset['train']['content'][0]

'Tapi enaknya akun baru Shopee itu belum ada biaya adminnya 😂 meskipun masih nyeseq ga bisa dibuka https://t.co/3pQxvTTlnI'

In [9]:
tokenizer.decode(tokenized_dataset['train']['input_ids'][0])

'tapi enak akun baru shope itu belum ada biaya adminya meski masih nyeseq ga bisa buka</s><pad><pad><pad><pad><pad><pad><pad>'

In [10]:
tokenizer.decode(tokenized_dataset['train']['labels'][0])

'(shopee,akun baru shopee,belum ada biaya adminnya,positive,payment)</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

# Model Training

In [32]:
# from transformers import Seq2SeqTrainer
# from transformers import DataCollatorForSeq2Seq

In [11]:
model = T5ForConditionalGeneration.from_pretrained(PRETRAINED_MODEL)
#model = BartForConditionalGeneration.from_pretrained(PRETRAINED_MODEL)
#model.resize_token_embeddings(len(tokenizer))

In [33]:
# data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)
# trainer = Seq2SeqTrainer(
#     model,
#     training_args,
#     train_dataset=tokenized_dataset["train"],
#     eval_dataset=tokenized_dataset["test"],
#     data_collator=data_collator,
# )

# trainer.train()

In [12]:
#training argument
training_args = Seq2SeqTrainingArguments(
    SAVE_PATH,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=0.0003,
    #weight_decay=0.01,
    resume_from_checkpoint=True,
    num_train_epochs=10,
    save_total_limit=2,
    seed=42
)

In [13]:
finetuner = FineTuner(model=model, save_path=SAVE_PATH, tokenizer=tokenizer, 
                      train_dataset=tokenized_dataset['train'], eval_dataset=tokenized_dataset['test'])

cuda:0


In [14]:
finetuner.fine_tune(training_args)

c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/2370 [00:00<?, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
                                                  
 10%|█         | 237/2370 [00:56<07:41,  4.62it/s]

{'eval_loss': 0.2741573750972748, 'eval_runtime': 0.5273, 'eval_samples_per_second': 189.638, 'eval_steps_per_second': 24.653, 'epoch': 1.0}


                                                    
 20%|██        | 474/2370 [02:10<07:24,  4.26it/s]

{'eval_loss': 0.24826639890670776, 'eval_runtime': 0.5883, 'eval_samples_per_second': 169.978, 'eval_steps_per_second': 22.097, 'epoch': 2.0}


 21%|██        | 500/2370 [02:39<08:09,  3.82it/s]  

...

{'eval_loss': 0.35090363025665283, 'eval_runtime': 0.5259, 'eval_samples_per_second': 190.143, 'eval_steps_per_second': 24.719, 'epoch': 9.0}


                                                   
100%|██████████| 2370/2370 [12:45<00:00,  4.72it/s]

{'eval_loss': 0.35784682631492615, 'eval_runtime': 0.5221, 'eval_samples_per_second': 191.525, 'eval_steps_per_second': 24.898, 'epoch': 10.0}


100%|██████████| 2370/2370 [13:11<00:00,  2.99it/s]


{'train_runtime': 791.6358, 'train_samples_per_second': 23.925, 'train_steps_per_second': 2.994, 'train_loss': 0.5070209623892096, 'epoch': 10.0}


In [15]:
model.save_pretrained(SAVE_PATH)

# Training Pipeline

In [2]:
#constant
SAVE_PATH = f'../models/preprocess_T5'
PRETRAINED_MODEL = "Wikidepia/IndoT5-base"
DATA_PATH = '../Data/quadruplet/quadruplet_annottated_sample_dataset_clean.csv'
#variable
max_length = 128
text_col = 'content'
label_col = 'quadruplet'
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)
#training argument
training_args = Seq2SeqTrainingArguments(
    SAVE_PATH,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    resume_from_checkpoint=True,
    num_train_epochs=20,
    save_total_limit=2,
)
#read dataset
raw_dataset = load_dataset('csv', data_files=DATA_PATH)
#model training
for i in range(4):
    preprocess_type = f'p0{i}'
    print(f"[INFO] preprocessing dataset for experiment {preprocess_type}")
    #preprocess dataset
    preprocessor = Preprocessor(f'{preprocess_type}', tokenizer, max_length, text_col, label_col)
    tokenized_dataset = raw_dataset.map(preprocessor.preprocess_dataset, batched=True, remove_columns=raw_dataset['train'].column_names)
    splitted_dataset = tokenized_dataset['train'].train_test_split(test_size=0.2, seed=42)
    print(f"[INFO] training model for experiment {preprocess_type}")
    model = T5ForConditionalGeneration.from_pretrained(PRETRAINED_MODEL)
    finetuner = FineTuner(model=model, save_path=f"{SAVE_PATH}_{preprocess_type}", tokenizer=tokenizer, 
                      train_dataset=splitted_dataset['train'], eval_dataset=splitted_dataset['test'])
    finetuner.fine_tune(training_args)

Found cached dataset csv (C:/Users/danendra/.cache/huggingface/datasets/csv/default-564c7144e159da05/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 420.36it/s]
Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-564c7144e159da05\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-0619265f74d98b9e.arrow
Loading cached split indices for dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-564c7144e159da05\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-5d08aae3a84c90c6.arrow and C:\Users\danendra\.cache\huggingface\datasets\csv\default-564c7144e159da05\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-4bae551a5fe5658f.arrow


[INFO] preprocessing dataset for experiment p0
[INFO] training model for experiment p0


c:\Users\danendra\anaconda3\envs\tesis\lib\site-packages\transformers\optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


cuda:0
cuda:0


  0%|          | 0/2260 [00:00<?, ?it/s]You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
                                                  
  5%|▌         | 113/2260 [00:21<06:26,  5.56it/s]

...

{'eval_loss': 0.6941794157028198, 'eval_runtime': 1.3898, 'eval_samples_per_second': 162.617, 'eval_steps_per_second': 20.867, 'epoch': 19.0}


100%|██████████| 2260/2260 [14:38<00:00,  5.71it/s]

{'eval_loss': 0.6934356689453125, 'eval_runtime': 1.2494, 'eval_samples_per_second': 180.889, 'eval_steps_per_second': 23.211, 'epoch': 20.0}


100%|██████████| 2260/2260 [15:05<00:00,  2.49it/s]


{'train_runtime': 905.9792, 'train_samples_per_second': 19.934, 'train_steps_per_second': 2.495, 'train_loss': 1.0219785707186808, 'epoch': 20.0}


# Inference

In [16]:
from src.inference import ModelInference
from src.preprocessor import Preprocessor
from transformers import (
    T5ForConditionalGeneration,
    AutoTokenizer,
)
from datasets import load_dataset

In [2]:
#constant
max_length = 30
text_col = 'content'
label_col = 'postprocess_quintuplet'
preprocess_type = 'p00'
SAVE_PATH = f'../models/quintuplet_t5_MLM-PT'
PRETRAINED_MODEL = "Wikidepia/IndoT5-base"#"../models/pt-indot5-MLM_PT" #"Wikidepia/IndoT5-base" 
TOKENIZER_PATH = "Wikidepia/IndoT5-base"
DATA_PATH = '../Data/quintuplet/quintuplet_postprocessed_1000.csv'
inference_length = 128

In [3]:
model = T5ForConditionalGeneration.from_pretrained(SAVE_PATH)
tokenizer = AutoTokenizer.from_pretrained(PRETRAINED_MODEL)

In [4]:
preprocessor = Preprocessor(preprocess_type, tokenizer, max_length, text_col, label_col)

In [5]:
raw_dataset = load_dataset('csv', data_files=DATA_PATH)
splitted_dataset = raw_dataset['train'].train_test_split(test_size=0.1, seed=42)
tokenized_dataset = splitted_dataset.map(preprocessor.preprocess_dataset, batched=True, remove_columns=raw_dataset['train'].column_names)

Found cached dataset csv (C:/Users/danendra/.cache/huggingface/datasets/csv/default-e45b5c90cd924547/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 500.04it/s]
Loading cached split indices for dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-e45b5c90cd924547\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-ff1816f6c2c3ba58.arrow and C:\Users\danendra\.cache\huggingface\datasets\csv\default-e45b5c90cd924547\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-64790f2f7fbaf197.arrow
Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-e45b5c90cd924547\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-b27f5c338b779ada.arrow
Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-e45b5c90cd924547\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853b

## pakai test data asli

In [18]:
model_inference = ModelInference(batch_size=8, dataset=tokenized_dataset['test'], model=model, 
                                 tokenizer=tokenizer, inference_len=512)
pred_text = model_inference.inference()

100%|██████████| 13/13 [00:07<00:00,  1.84it/s]


In [19]:
test_dataset = splitted_dataset['test']
test_dataset = test_dataset.add_column(f'{preprocess_type}_model_prediction', pred_text)
test_dataset.to_csv('../Data/quintuplet/test_data_predictions.csv')

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 16.52ba/s]


59610

## buat dummy data

In [29]:
#membuat dummy dataset
# dummy_dataset = splitted_dataset['train'].train_test_split(test_size=400, seed=42)
# test_dataset = dummy_dataset['test'].train_test_split(test_size=200, seed=42)
# test_dataset['train'].to_csv('../Data/quintuplet/val_data.csv') 
# test_dataset['test'].to_csv('../Data/quintuplet/test_data.csv') 
#.map(preprocessor.preprocess_dataset, batched=True, remove_columns=raw_dataset['train'].column_names)

Loading cached split indices for dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-7c3f713988208aa5\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-2c42dbc24aa38187.arrow and C:\Users\danendra\.cache\huggingface\datasets\csv\default-7c3f713988208aa5\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-f5003ba921be7c84.arrow
Loading cached split indices for dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-7c3f713988208aa5\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-52fb2de3f9cb2504.arrow and C:\Users\danendra\.cache\huggingface\datasets\csv\default-7c3f713988208aa5\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-29b8e9a42cc5be2e.arrow
Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 117.52ba/s]


105907

In [17]:
#raw dataset
val_dataset = load_dataset('csv', data_files='../Data/quintuplet/val_data.csv')
test_dataset = load_dataset('csv', data_files='../Data/quintuplet/test_data.csv')
#tokenized dataset
tokenized_val_dataset = val_dataset.map(preprocessor.preprocess_dataset, batched=True, remove_columns=val_dataset['train'].column_names)
tokenized_test_dataset = test_dataset.map(preprocessor.preprocess_dataset, batched=True, remove_columns=test_dataset['train'].column_names)

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 51.24it/s]


Dataset csv downloaded and prepared to C:/Users/danendra/.cache/huggingface/datasets/csv/default-b8dcae55ca6bd5c3/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1. Subsequent calls will reuse this data.


100%|██████████| 1/1 [00:00<00:00, 49.95it/s]
Found cached dataset csv (C:/Users/danendra/.cache/huggingface/datasets/csv/default-ae888e11fe36b0d0/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
100%|██████████| 1/1 [00:00<00:00, 500.81it/s]
Loading cached processed dataset at C:\Users\danendra\.cache\huggingface\datasets\csv\default-ae888e11fe36b0d0\0.0.0\6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1\cache-71127a19e7f0cb1f.arrow


In [18]:
model_inference = ModelInference(batch_size=8, dataset=tokenized_val_dataset['train'], model=model, 
                                 tokenizer=tokenizer, inference_len=512)
pred_text = model_inference.inference()

  0%|          | 0/25 [00:00<?, ?it/s]

100%|██████████| 25/25 [00:17<00:00,  1.47it/s]


In [19]:
val_dataset['train'] = val_dataset['train'].add_column(f'{experiment_type}_model_prediction', pred_text)
val_dataset['train'].to_csv('../Data/quintuplet/val_data.csv')

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 34.90ba/s]


509672

## using train data

In [24]:
model_inference = ModelInference(batch_size=8, dataset=tokenized_dataset['train'], model=model, 
                                 tokenizer=tokenizer, inference_len=512)
pred_text = model_inference.inference()

  3%|▎         | 8/237 [00:06<02:54,  1.31it/s]


KeyboardInterrupt: 

In [64]:
train_dataset = splitted_dataset['train']
train_dataset = train_dataset.add_column(f'{preprocess_type}_model_prediction', pred_text)
train_dataset.to_csv('../Data/quintuplet/model_1000-data_predictions_train.csv')

Creating CSV from Arrow format: 100%|██████████| 1/1 [00:00<00:00, 39.96ba/s]


543439

# Evaluate

In [20]:
import pandas as pd
from src.evaluator import Evaluator
from src.postprocessor import PostProcessor
from src.utils import extract_triplet

In [21]:
df = pd.read_csv('../Data/quintuplet/val_data.csv')
df['postprocess_quintuplet'] = df['postprocess_quintuplet'].apply(lambda x:x.lower())
df.head(3)

,aoriginal_id,content,final_sentiment,baseline_aspect_category,tweet_type,clean_tweet,label,corrected_label,keterangan,quintuplet_label,postprocess_quintuplet,p00_model_prediction,p01_model_prediction,p02_model_prediction,p03_model_prediction,p02_bart_model_prediction,p20_model_prediction,p21_model_prediction,p22_model_prediction
0,1648822375587381249,"@JNE_ID JT78364921062 ,, Ma,af no resi itu ,, ...",negative,price; produk;,SUBJECTIVE,"JT78364921062 ,, Ma,af no resi itu ,, order d...","(shopee, order, blm sampe ke alamat penerimany...",NaN,NaN,"(shopee, order, blm sampe ke alamat penerimany...","(shopee,order,blm sampe ke alamat penerimanya,...","(shopee,order,blm sampe ke alamat penerimanya,...","(shopee,order,blm sampe ke alamat penerimanya,...","(shopee,order,blm sampe ke alamat penerimanya,...","(shopee,order,blm sampe ke alamat penerimanya,...","(shopee,shopee express,cepet,positive,deliver...","(shopee,order,blm sampe ke alamat penerimanya,...","(shopee,order,blm sampe ke alamat penerimanya,...","(shopee,order,blm sampe ke alamat penerimanya,..."
1,1639829371166015489,@sbtcon Jodantae sama shopee express soalnya a...,negative,produk;,SUBJECTIVE,Jodantae sama shopee express soalnya abang ab...,"(shopee, shopee express, cepet, positive, deli...",NaN,NaN,"(shopee, shopee express, cepet, positive, deli...","(shopee,shopee express,cepet,positive,delivery)","(shopee,shopee express,cepet,positive,delivery)","(shopee,shopee express,cepet,positive,delivery)","(shopee,shopee express,cepet,positive,delivery)","(shopee,shopee express,cepet,positive,delivery)","(shopee,shopee express,cepet,positive,deliver...","(shopee,shopee express,cepet,positive,delivery)","(shopee,shopee express,cepet,positive,delivery)","(shopee,shopee express,cepet,positive,delivery)"
2,1640221429517156352,Berubah ketika barang sampai.yg tadinya ada ja...,negative,produk;,SUBJECTIVE,Berubah ketika barang sampai.yg tadinya ada ja...,"(tokopedia, invoice, bisa ubah invoice seenak ...",NaN,NaN,"(tokopedia, invoice, bisa ubah invoice seenak ...","(tokopedia,invoice,bisa ubah invoice seenak me...","(_,barang,sampai skrng,negative,delivery)","(_,barang,ubah barang dari yg tadinya jadi ila...","(tokopedia,barang,sampai yg tadi ada jd ilang,...","(tokopedia,pengubah barang,jadi lenyap,negativ...","(shopee,shopee express,cepet,positive,deliver...","(tokopedia,barang,sampai yg tadi ada jadi leny...","(tokopedia,barang,tiba2 jadi lenyap,negative,d...","(tokopedia,barang,nyasar,negative,delivery)"


In [22]:
postprocessor = PostProcessor(use_postprocess=False)
evaluator = Evaluator(task_type='quintuplet', postprocessor=postprocessor)

In [23]:
raw_scores, all_labels, all_preds = evaluator.evaluate(pred_seqs=df[f'{experiment_type}_model_prediction'],
                   gold_seqs=df['postprocess_quintuplet'])

100%|██████████| 200/200 [00:00<00:00, 100054.96it/s]


In [25]:
raw_scores

{'precision': 0.858, 'recall': 0.85, 'f1': 0.854}

train score

In [72]:
df = pd.read_csv('../Data/quintuplet/model_1000-data_predictions_train.csv')
df['postprocess_quintuplet'] = df['postprocess_quintuplet'].apply(lambda x:x.lower())
df.head(3)

,aoriginal_id,content,final_sentiment,baseline_aspect_category,tweet_type,clean_tweet,label,corrected_label,keterangan,quintuplet_label,postprocess_quintuplet,p00_model_prediction
0,1647261067511496704,eh mau lebaran kyk gini ekspedisi kan lagi hec...,negative,produk;,SUBJECTIVE,eh mau lebaran kyk gini ekspedisi kan lagi hec...,"(shopee, shopee xpress, ekpsedisi kan lagi hec...",NaN,NaN,"(shopee, shopee xpress, ekpsedisi kan lagi hec...","(shopee,shopee xpress,ekpsedisi kan lagi hecti...","(shopee,shopee xpress,ekspedisi kan lagi hecti..."
1,1646546316204597248,@tanyakanrl asli pernah lewat beranda tbtb ada...,negative,delivery; website&apps; produk;,SUBJECTIVE,asli pernah lewat beranda tbtb ada yg jual ko...,"(shopee, konten dewasa, parah si kok bisa lolo...",NaN,NaN,"(shopee, konten dewasa, parah si kok bisa lolo...","(shopee,konten dewasa,parah si kok bisa lolos ...","(shopee,konten dewasa,parah si kok bisa lolos ..."
2,1647243563443388416,"Adeuh, ini barang gua dri shopee blm nyampai2 ...",negative,delivery; customerservice; produk;,SUBJECTIVE,"Adeuh, ini barang gua dri shopee blm nyampai2 ...","(shopee, barang, blm nyampai2 jg woy, negative...",NaN,NaN,"(shopee, barang, blm nyampai2 jg woy, negative...","(shopee,barang,blm nyampai2 jg woy,negative,de...","(shopee,barang,blm nyampai2 jg woy,negative,de..."


In [73]:
raw_scores, all_labels, all_preds = evaluator.evaluate(pred_seqs=df['p00_model_prediction'],
                   gold_seqs=df['postprocess_quintuplet'])

100%|██████████| 930/930 [00:00<00:00, 109199.15it/s]


In [74]:
raw_scores

{'precision': 0.6540483701366983,
 'recall': 0.6032977691561591,
 'f1': 0.627648839556004}